# The Main_Input Class 

This notebook demonstrates how to use the main_input class.   The main_input class was developed primarily to assist with parameter-space studies, but it may be useful in other situations as well.  We begin with an overview of how to work with main_input objects in python, and then provide an example of how they might be used to help initiate a parameter space study.   Before running this notebook, please copy 'Rayleigh/post_processing/benchmark_diagnostics_input' to the directory from within which you are running this notebook.  Within that directory, you should find a "jobinfo.txt" and "main_input" file that will be used in the examples below.


Documentation for all methods and attributes of this class can be accessed via Python's help function, as illustrated below following the import.

In [ ]:
from rayleigh_diagnostics import main_input
help(main_input)

## Initialization

To initialize an instance of the main_input class, we must provide a main_input or jobinfo.txt file.   That file will be used to define and initialize the object's internal namelist structure.  An important point to keep in mind is that the namelist structure derives entirely from the main_input file.  Values omitted in the main_input file are not defined for the main_input class.  There are workarounds for this illustrated below.   To begin, let's read in a main_input file (one is provided within the directory that containst his notebook) and view its contents. The most straightforward way to do this is by using Python's print function, which will display all namelist entries that have been assigned a value.

In [ ]:
mi = main_input('main_input')
print(mi) 

## The write and set Methods

Python's print function calls the main_input class's *write* method with no optional keywords specified.  For the curious, this is done by defining the  *__repr__* for the main_input class.  By using the write method directly, we can have finer-grained control over the output.  Specifically, we can:

1.  Print a single namelist
2.  Display nullstring values (e.g., benchmark_mode in the above example) using the verbose optional parameter.
3.  Control the precision of floating-point output using the ndecimal optional parameter.
4.  Write a new main_input file using the *file* optional parameter.

This functionality is illustrated below, in combination with the set method, which is used to assign new namelist values.  Note that verbose does not impact values written to a new main_input file; nullstring values are ignored.  Examine the contents of *new_main_input* after running the cell below.  Nprow is omitted entirely, and aspect ratio has 2 digits after the decimal.  Anything from the original file is stored as a string, including numerical values, and remains unmodified.

In [ ]:
mi.write(namelist='problemsize')  # write only the problemsize namelist

mi.set(nml='problemsize',var='nprow',val='') # set nprow to the null string
mi.set(nml='problemsize',var='aspect_ratio',val=1/3) # set aspect ratio to a float

#Omit null values
print('\n\nDisplaying problemsize namelist without null values')
mi.write(namelist='problemsize')

#Include null values and use 3 decimal places (only affects floats)
#Note that shell_depth is stored as a string, and so its precision remains unmodified
print('\n\nDisplaying problemsize including null values and with 3 decimal places')
mi.write(namelist='problemsize',verbose=True, ndecimal=3)


# Attempting to assign a value to an undefined namelist variable
# will generate an error
mi.set(nml='problemsize',var='l_max',val=127)

# If you are sure you didn't mispell the variable and want to add it, use "force=True"
mi.set(nml='problemsize',var='l_max',val=127, force=True)
mi.write(namelist='problemsize')

#Finally, we can create a new main_input file using the write method
mi.write(file='new_main_input', ndecimal=2) # write-to-file with 2 digits for float values

## Namelist Values:  the *vals* dictionary

Namelist variable names and values read from main_input are stored in the *vals* dictionary associated with the main_input class.  Their values can also be modified by manipulating this dictionary directly as shown below.  Note that when specifying a namelist name, the "_namelist" is omitted.    Using the set method is recommended, as it checks the validity of variable names.

Note that all values read in from a main_input file are stored as strings.  This is why shell_depth was unaffected by value of ndecimal in the example above.  Keys in the *vals* dictionary may be assigned a numerical value.  When *print* is called, this value is converted to a string.  All floats are converted to scientific notation upon output (where to screen or to file).  The example below modifies the value of shell_depth such that it is stored as a float.


**Important:** When working directly with the vals dictionary, use all lower case for the key names.  The set method is case agnostic.

In [ ]:
sd = mi.vals['problemsize']['shell_depth']
print('type(shell_depth) from main_input: ', type(sd))


mi.vals['problemsize']['shell_depth'] = 2.0
sd = mi.vals['problemsize']['shell_depth']
print('type(shell_depth) from main_input: ', type(sd))

#Now shell_depth will be displayed with the desired precision
mi.write(namelist='problemsize', ndecimal=8)

## The Read and Unset Methods

After a main_input object is instantiated, we can subsequently modify its structure and values using the contents of another file via the *read* method.  We can also erase all values in the vals dictionary entirely, while maintaining the dictionary/file structure using the *unset* method.  The example below illustrates this functionality using jobinfo.txt.  That file is generated by Rayleigh at runtime, and it contains a list of all namelist variables and associated values.  Jobinfo.txt can be useful if you want to access a list of all available namelist variables within your Python routines.

In [ ]:
mi = main_input('jobinfo.txt')
mi.unset()
mi.read('main_input')
mi.write(verbose=True)

## Use Case:  Parameter Space Study 


Finally, we illustrate how to set up a small parameter-space study using the main_input class.  Typically, only a few values in main_input will change from model to model.  In particular, the list of outputs and initial conditions will likely remain unmodified.  First, let us specify set of physical parameters and simulation resolution sizes.  Note that the combination of supercriticality and resolution used here is for illustration purposes only.  These values are purposefully unrealistic. 

In [ ]:
import os
ra_values = [1e1, 1e2]
ek_values = [1e-3, 1e-4]
ra_res_nr = [48, 96]
ra_res_ntheta = [48, 96]

base_dir = 'parameter_study'  # top-level directory within which each simulation directory will reside
try:
    os.mkdir(base_dir)
except:
    pass # avoid error message if directory exists

Next, read in a main_input file that you wish to use as a template.   For this example, we will use the template provided with this notebook.   This particular file has benchmarking turned on.  Before proceeding, we turn benchmarking off and use a random thermal perturbations about a conductive background for the initial conditions instead.

In [ ]:
mi = main_input('jobinfo.txt')
mi.unset()
mi.read('main_input')

mi.vals['physical_controls']['benchmark_mode'] = ""
mi.vals['physical_controls']['benchmark_integration_interval'] = ""
mi.vals['physical_controls']['benchmark_report_interval'] = ""
mi.vals['initial_conditions']['init_type'] = 7
mi.vals['initial_conditions']['temp_amp'] = 0.001
mi.vals['initial_conditions']['conductive_profile'] = '.true.'
print('This is the main_input template: \n\n')
print(mi)

Finally, we generate input files for our small parameter-space study.  Examine the contents of parameter_study/model_X/main_input to verify the changes.

In [ ]:
mnum = 1
for i, ra in enumerate(ra_values):
    for ek in ek_values:
        ra_str = 'Ra_'+"{:.2e}".format(ra)
        ek_str = 'Ek_'+"{:.2e}".format(ek)
        sim_name = 'model '+str(mnum)
        sim_dir =  base_dir+'/model_'+str(mnum)
        print(sim_name,':','Ra =',ra_str, ',', 'Ek =', ek_str)
        
        try:
            os.mkdir(sim_dir)
        except:
            pass

        input_file = sim_dir+'/main_input'
        mi.vals['problemsize']['n_r'] = ra_res_nr[i]
        mi.vals['problemsize']['n_theta'] = ra_res_ntheta[i]
        mi.vals['reference']['rayleigh_number'] = ra
        mi.vals['reference']['ekman_number'] = ek
        mi.write(file=input_file)
        mnum+=1